## How to create a closed-loop optimisation using a machine learning model as the benchmark function

In [ ]:
# Import the variable, objectives, sampler, acquisition function, and the optimisation classes
from nemo_bo.opt.variables import ContinuousVariable, VariablesList
from nemo_bo.opt.objectives import RegressionObjective, ObjectivesList
from nemo_bo.acquisition_functions.expected_improvement.expected_improvement import (
    ExpectedImprovement,
)
from nemo_bo.opt.samplers import LatinHyperCubeSampling
from nemo_bo.opt.optimisation import Optimisation

In [ ]:
# Create the variable objects
var1 = ContinuousVariable(name="variable1", lower_bound=0.0, upper_bound=100.0)
var2 = ContinuousVariable(name="variable2", lower_bound=0.0, upper_bound=100.0)
var_list = VariablesList([var1, var2])

In [ ]:
# Create the objective objects
obj1 = RegressionObjective(name="objective1", obj_max_bool=True, lower_bound=0.0, upper_bound=100.0)
obj2 = RegressionObjective(name="objective2", obj_max_bool=False, lower_bound=0.0, upper_bound=100.0)
obj_list = ObjectivesList([obj1, obj2])

In [ ]:
# Instantiate the sampler
sampler = LatinHyperCubeSampling()

In [ ]:
# Instantiate the acquisition function
acq_func = ExpectedImprovement(num_candidates=4)

### Setting up the benchmark

As the benchmark represents the optimisation problem, the corresponding variables and objectives will be the same as those defined above. However, copies of the variable and objective objects are used to instatiate the `ModelBenchmark` class to ensure that they are not linked.

In [ ]:
import copy
from nemo_bo.opt.benchmark import ModelBenchmark

# Create the VariablesList for the benchmark function
benchmark_var_list = VariablesList([copy.copy(var1), copy.copy(var2)])
benchmark_obj_list = ObjectivesList([copy.copy(obj1), copy.copy(obj2)])

# Instantiate the ModelBenchmark class to be the benchmark function
benchmark = ModelBenchmark(benchmark_var_list, benchmark_obj_list)

# Fit the benchmark function using a X and Y arrays as the training data that is representative of the variable and objective space
# X_benchmark and Y_benchmark arrays represent hypothetical datasets that are good for training the benchmark function
benchmark.fit(X=X_benchmark, Y=Y_benchmark)

In [ ]:
# Set up the optimisation instance whilst passing benchmark function
optimisation = Optimisation(var_list, obj_list, acq_func, sampler=sampler, benchmark_func=benchmark)

### Starting the optimisation run with the benchmark function

When using a benchmark function in an optimisation, X and Y arrays do not need to be passed. The training set will be automatically created, where the number of samples is related to the sampler type chosen and the number of variables (`number of samples = (2 * number of variables) + 2`)

When the plot_progress keyword argument is True, a pareto plot will be created at every iteration.

In [ ]:
# Start the optimisation using the convenient run function that will run for the specified number of iterations
optimisation_data = optimisation.run(number_of_iterations=50, plot_progress=True)